In [23]:
import jupyter_Importer
from MLModels_FeatureImportance import featureImportance_linearRegression
from MLModels_FeatureImportance import featureImportance_permutation
from MLModels_FeatureImportance import featureImportance_DTree

import pandas as pd
import numpy as np
import math
from sklearn import preprocessing

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split

In [24]:
dataset = pd.read_csv("Occupancy.csv", names = ['x' + str(i) for i in range(1, 7)] + ['y'], sep = ",")
dataset.head()

,x1,x2,x3,x4,x5,x6,y
0,date,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
1,2015-02-02 14:19:00,23.7,26.272,585.2,749.2,0.0047641630241641,1
2,2015-02-02 14:19:59,23.718,26.29,578.4,760.4,0.0047726609921251,1
3,2015-02-02 14:21:00,23.73,26.23,572.666666666667,769.666666666667,0.0047651525524654,1
4,2015-02-02 14:22:00,23.7225,26.125,493.75,774.75,0.0047437733559968,1


In [25]:
dataset_light_co2 = dataset.iloc[1:,3:5]
dataset_light_co2['y'] = dataset.iloc[1:,6:7]
dataset_light_co2.columns = ['x1','x2','y']
dataset_light_co2['x1'] = dataset_light_co2['x1'].apply(lambda x : float(x))
dataset_light_co2['x2'] = dataset_light_co2['x2'].apply(lambda x : float(x))
dataset_light_co2['y'] = dataset_light_co2['y'].apply(lambda x : int(x))
dataset_light_co2

,x1,x2,y
1,585.200000,749.200000,1
2,578.400000,760.400000,1
3,572.666667,769.666667,1
4,493.750000,774.750000,1
5,488.600000,779.000000,1
...,...,...,...
20556,429.750000,1505.250000,1
20557,423.500000,1514.500000,1
20558,423.500000,1521.500000,1
20559,418.750000,1632.000000,1


In [26]:
subset = dataset_light_co2.sample(frac=0.2,replace=False,random_state=43)
dataset_light_co2 = dataset_light_co2.append(subset)
dataset_light_co2 = dataset_light_co2.drop_duplicates(subset=['x1','x2','y'],keep=False)

subsample_x = subset.iloc[:,:-1].values
subsample_y = subset.iloc[:,-1].values

#from 0-24 means angle
error_a = []
error_b = []
for i in range(0,25) :
    error_a.append(i)
    error_b.append(24-i)

print(error_a,error_b)
print(dataset_light_co2.info(),subset.info())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24] [24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6757 entries, 1 to 20560
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x1      6757 non-null   float64
 1   x2      6757 non-null   float64
 2   y       6757 non-null   int64  
dtypes: float64(2), int64(1)
memory usage: 211.2 KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4112 entries, 8855 to 4674
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x1      4112 non-null   float64
 1   x2      4112 non-null   float64
 2   y       4112 non-null   int64  
dtypes: float64(2), int64(1)
memory usage: 128.5 KB
None None


In [27]:
importanceReg,RegCoef = featureImportance_linearRegression(subsample_x,subsample_y)
model = KNeighborsClassifier(n_neighbors=5,algorithm="brute")  
model.fit(subsample_x,subsample_y)  
importancePermu,PermuCoef = featureImportance_permutation(model,subsample_x,subsample_y)
importanceDTree,DTreeCoef = featureImportance_DTree(subsample_x,subsample_y)
print("RegCoef : " +str(RegCoef))
print("importanceReg : "+str(importanceReg))
print("PermuCoef : "+str(PermuCoef))
print("importancePermu : "+str(importancePermu))
print("DTreeCoef : " +str(DTreeCoef))
print("importanceDTree : "+str(importanceDTree))

RegCoef : [0.001715115813006853, 0.0001540538185663689]
importanceReg : [0.9175816812106525, 0.08241831878934744]
PermuCoef : [0.30398832684824906, 0.05972762645914398]
importancePermu : [0.8357849692431131, 0.16421503075688687]
DTreeCoef : [0.9613610903214472, 0.03863890967855284]
importanceDTree : [0.9613610903214472, 0.03863890967855284]


In [28]:
def df_2feature_be_generates(BE,i,n,importance=importanceReg,df=dataset_light_co2.copy()) :#i = 1 for brute force iteration,n = 1for negative importance
    temp = dataset_light_co2.copy()
    angle = 0 #0 for normal case ,1 for adding error with cos sin
    #switch = {1:func1,2:func2}
    #temp = df.copy()
    df_2feature_be = pd.DataFrame.to_numpy(dataset_light_co2)
    #df_2feature_be = pd.DataFrame.to_numpy(df)
    
    if i >= 0 and n == 0 :
        angle = 1
        e1 = temp['x1'].mean()*BE*math.cos((error_a[i]/(error_a[i]+error_b[i])))
        e2 = temp['x2'].mean()*BE*math.sin((error_b[i]/(error_a[i]+error_b[i])))
    elif i == -1 and n == 1:
        angle = 0
        e1 = temp['x1'].mean()*BE*importance[0]
        e2 = temp['x2'].mean()*BE*importance[1]
    elif i == -1 and n == 0 :
        angle = 0
        e1 = temp['x1'].mean()*BE*importance[1]
        e2 = temp['x2'].mean()*BE*importance[0]
    

    if angle == 0:
        for idx,cell in temp['x1'].iteritems():
            temp['x1'][idx] += e1
        temp = temp.to_numpy()
        df_2feature_be = np.vstack((df_2feature_be,temp))
        #print(df_2feature_be.shape)
        temp = dataset_light_co2.copy()

        for idx,cell in temp['x1'].iteritems():
            temp['x1'][idx] = temp['x1'][idx] - e1
        temp = temp.to_numpy()
        df_2feature_be = np.vstack((df_2feature_be,temp))
        #print(df_2feature_be.shape)
        temp = dataset_light_co2.copy()

        for idx,cell in temp['x2'].iteritems():
            temp['x2'][idx] += e2
        temp = temp.to_numpy()
        df_2feature_be = np.vstack((df_2feature_be,temp))
        #print(df_2feature_be.shape)
        temp = dataset_light_co2.copy()

        for idx,cell in temp['x2'].iteritems():
            temp['x2'][idx] = temp['x2'][idx] - e2
        temp = temp.to_numpy()
        df_2feature_be = np.vstack((df_2feature_be,temp))
        #print(df_2feature_be.shape)
        temp = dataset_light_co2.copy()

        df_2feature_be = pd.DataFrame(df_2feature_be)
        #print(df_2feature_be)

    elif angle == 1:
        for idx,cell in temp['x1'].iteritems():
            temp['x1'][idx] += e1
        temp = temp.to_numpy()
        df_2feature_be = np.vstack((df_2feature_be,temp))
        #print(df_2feature_be.shape)
        temp = dataset_light_co2.copy()

        for idx,cell in temp['x2'].iteritems():
            temp['x2'][idx] += e2
        temp = temp.to_numpy()
        df_2feature_be = np.vstack((df_2feature_be,temp))
        #print(df_2feature_be.shape)
        temp = dataset_light_co2.copy()
        
        df_2feature_be = pd.DataFrame(df_2feature_be)

    

    return df_2feature_be

In [29]:
#using importanceReg as error allocation rate to predict by svm
BE = 0.2
df_2feature_be = df_2feature_be_generates(BE,-1,0,importanceReg)
x_be = df_2feature_be.iloc[:, :-1].values
y_be = df_2feature_be.iloc[:,-1].values
X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
clf_BE = SVC(kernel='linear',C=1,gamma='auto')
clf_BE.fit(X_train, y_train)
y_BE_predict = clf_BE.predict(X_test)
result_BE_train = clf_BE.score(X_train, y_train)
result_BE_test = clf_BE.score(X_test, y_test)
subset_raw = clf_BE.score(subsample_x, subsample_y)
print('BESVM Accuracy (on training) = with x1 bounded error ('+str(BE)+'%*'+str(importanceReg[1])+'),x2 with error ('+str(BE)+'%*'+str(importanceReg[0])+') : '+str(result_BE_train))
print('BESVM Accuracy (on testing) = with x1 bounded error ('+str(BE)+'%*'+str(importanceReg[1])+'),x2 with error ('+str(BE)+'%*'+str(importanceReg[0])+') : '+str(result_BE_test))
print('BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error '+str(subset_raw))


d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to b

BESVM Accuracy (on training) = with x1 bounded error (0.2%*0.08241831878934744),x2 with error (0.2%*0.9175816812106525) : 0.9728799763208524
BESVM Accuracy (on testing) = with x1 bounded error (0.2%*0.08241831878934744),x2 with error (0.2%*0.9175816812106525) : 0.9738049430220512
BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9878404669260701


In [30]:
#using importancePermu as error allocation rate to predict by svm
df_2feature_be = df_2feature_be_generates(BE,-1,0,importancePermu)
x_be = df_2feature_be.iloc[:, :-1].values
y_be = df_2feature_be.iloc[:,-1].values
X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
clf_BE = SVC(kernel='linear',C=1,gamma='auto')
clf_BE.fit(X_train, y_train)
y_BE_predict = clf_BE.predict(X_test)
result_BE_train = clf_BE.score(X_train, y_train)
result_BE_test = clf_BE.score(X_test, y_test)
subset_raw = clf_BE.score(subsample_x, subsample_y)
print('BESVM Accuracy (on training) = with x1 bounded error ('+str(BE)+'%*'+str(importancePermu[1])+'),x2 with error ('+str(BE)+'%*'+str(importancePermu[0])+') : '+str(result_BE_train))
print('BESVM Accuracy (on testing) = with x1 bounded error ('+str(BE)+'%*'+str(importancePermu[1])+'),x2 with error ('+str(BE)+'%*'+str(importancePermu[0])+') : '+str(result_BE_test))
print('BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error '+str(subset_raw))



d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to b

BESVM Accuracy (on training) = with x1 bounded error (0.2%*0.16421503075688687),x2 with error (0.2%*0.8357849692431131) : 0.9730649696610922
BESVM Accuracy (on testing) = with x1 bounded error (0.2%*0.16421503075688687),x2 with error (0.2%*0.8357849692431131) : 0.9738049430220512
BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9878404669260701


In [31]:
#brute force allocation
#brute force from 15-360 degree using svm
BE = 0.2
for i in range(25) :
    df_2feature_be = df_2feature_be_generates(BE,i,0)
    x_be = df_2feature_be.iloc[:, :-1].values
    y_be = df_2feature_be.iloc[:,-1].values
    X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
    clf_BE = SVC(kernel='linear',C=1,gamma='auto')
    clf_BE.fit(X_train, y_train)
    y_BE_predict = clf_BE.predict(X_test)
    
    result_BE_train = clf_BE.score(X_train, y_train)
    result_BE_test = clf_BE.score(X_test, y_test)
    subset_raw = clf_BE.score(subsample_x, subsample_y)
    print('BESVM Accuracy (on training) = with x1 bounded error ('+str(BE)+'%*'+str((error_a[i]/(error_a[i]+error_b[i])))+'),x2 with error ('+str(BE)+'%*'+str((error_b[i]/(error_a[i]+error_b[i])))+') : '+str(result_BE_train))
    print('BESVM Accuracy (on testing) = with x1 bounded error ('+str(BE)+'%*'+str((error_a[i]/(error_a[i]+error_b[i])))+'),x2 with error ('+str(BE)+'%*'+str((error_b[i]/(error_a[i]+error_b[i])))+') : '+str(result_BE_test))
    print('BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error '+str(subset_raw))
    

d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


BESVM Accuracy (on training) = with x1 bounded error (0.2%*0.0),x2 with error (0.2%*1.0) : 0.9621978293043907
BESVM Accuracy (on testing) = with x1 bounded error (0.2%*0.0),x2 with error (0.2%*1.0) : 0.9595561035758323
BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9764105058365758


d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


BESVM Accuracy (on training) = with x1 bounded error (0.2%*0.041666666666666664),x2 with error (0.2%*0.9583333333333334) : 0.9673778983719783
BESVM Accuracy (on testing) = with x1 bounded error (0.2%*0.041666666666666664),x2 with error (0.2%*0.9583333333333334) : 0.9676942046855733
BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9880836575875487


d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


...

BESVM Accuracy (on training) = with x1 bounded error (0.2%*0.75),x2 with error (0.2%*0.25) : 0.9689812530833745
BESVM Accuracy (on testing) = with x1 bounded error (0.2%*0.75),x2 with error (0.2%*0.25) : 0.9699136868064119
BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9880836575875487


d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


BESVM Accuracy (on training) = with x1 bounded error (0.2%*0.7916666666666666),x2 with error (0.2%*0.20833333333333334) : 0.9660828811050814
BESVM Accuracy (on testing) = with x1 bounded error (0.2%*0.7916666666666666),x2 with error (0.2%*0.20833333333333334) : 0.9654747225647349
BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9824902723735408


d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


BESVM Accuracy (on training) = with x1 bounded error (0.2%*0.8333333333333334),x2 with error (0.2%*0.16666666666666666) : 0.969597927972373
BESVM Accuracy (on testing) = with x1 bounded error (0.2%*0.8333333333333334),x2 with error (0.2%*0.16666666666666666) : 0.9704069050554871
BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9880836575875487


#Knn

In [32]:
#using importanceReg as error allocation rate to predict by knn
df_2feature_be = df_2feature_be_generates(BE,-1,0,importanceReg)
x_be = df_2feature_be.iloc[:, :-1].values
y_be = df_2feature_be.iloc[:,-1].values
X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
KNN_BE = KNeighborsClassifier()
KNN_BE.fit(X_train, y_train)
y_BE_predict = KNN_BE.predict(X_test)
result_BE_train = KNN_BE.score(X_train, y_train)
result_BE_test = KNN_BE.score(X_test, y_test)
subset_raw = KNN_BE.score(subsample_x, subsample_y)
print('BEKNN Accuracy (on training) = with x1 bounded error ('+str(BE)+'%*'+str(importanceReg[1])+'),x2 with error ('+str(BE)+'%*'+str(importanceReg[0])+') : '+str(result_BE_train))
print('BEKNN Accuracy (on testing) = with x1 bounded error ('+str(BE)+'%*'+str(importanceReg[1])+'),x2 with error ('+str(BE)+'%*'+str(importanceReg[0])+') : '+str(result_BE_test))
print('BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error '+str(subset_raw))


d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to b

BEKNN Accuracy (on training) = with x1 bounded error (0.2%*0.08241831878934744),x2 with error (0.2%*0.9175816812106525) : 0.9794657392333876
BEKNN Accuracy (on testing) = with x1 bounded error (0.2%*0.08241831878934744),x2 with error (0.2%*0.9175816812106525) : 0.9748409057273938
BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9883268482490273


In [33]:
#using importancePermu as error allocation rate to predict by knn
df_2feature_be = df_2feature_be_generates(BE,-1,0,importancePermu)
x_be = df_2feature_be.iloc[:, :-1].values
y_be = df_2feature_be.iloc[:,-1].values
X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
KNN_BE = KNeighborsClassifier()
KNN_BE.fit(X_train, y_train)
y_BE_predict = KNN_BE.predict(X_test)
result_BE_train = KNN_BE.score(X_train, y_train)
result_BE_test = KNN_BE.score(X_test, y_test)
subset_raw = KNN_BE.score(subsample_x, subsample_y)
print('BEKNN Accuracy (on training) = with x1 bounded error ('+str(BE)+'%*'+str(importancePermu[1])+'),x2 with error ('+str(BE)+'%*'+str(importancePermu[0])+') : '+str(result_BE_train))
print('BEKNN Accuracy (on testing) = with x1 bounded error ('+str(BE)+'%*'+str(importancePermu[1])+'),x2 with error ('+str(BE)+'%*'+str(importancePermu[0])+') : '+str(result_BE_test))
print('BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error '+str(subset_raw))


d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to b

BEKNN Accuracy (on training) = with x1 bounded error (0.2%*0.16421503075688687),x2 with error (0.2%*0.8357849692431131) : 0.977652804499038
BEKNN Accuracy (on testing) = with x1 bounded error (0.2%*0.16421503075688687),x2 with error (0.2%*0.8357849692431131) : 0.9749889003995856
BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9883268482490273


In [34]:
#using importanceDTree as error allocation rate to predict by knn
df_2feature_be = df_2feature_be_generates(BE,-1,0,importanceDTree)
x_be = df_2feature_be.iloc[:, :-1].values
y_be = df_2feature_be.iloc[:,-1].values
X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
KNN_BE = KNeighborsClassifier()
KNN_BE.fit(X_train, y_train)
y_BE_predict = KNN_BE.predict(X_test)
result_BE_train = KNN_BE.score(X_train, y_train)
result_BE_test = KNN_BE.score(X_test, y_test)
subset_raw = KNN_BE.score(subsample_x, subsample_y)
print('BEKNN Accuracy (on training) = with x1 bounded error ('+str(BE)+'%*'+str(importanceDTree[1])+'),x2 with error ('+str(BE)+'%*'+str(importanceDTree[0])+') : '+str(result_BE_train))
print('BEKNN Accuracy (on testing) = with x1 bounded error ('+str(BE)+'%*'+str(importanceDTree[1])+'),x2 with error ('+str(BE)+'%*'+str(importanceDTree[0])+') : '+str(result_BE_test))
print('BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error '+str(subset_raw))


d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to b

BEKNN Accuracy (on training) = with x1 bounded error (0.2%*0.03863890967855284),x2 with error (0.2%*0.9613610903214472) : 0.9825736273494154
BEKNN Accuracy (on testing) = with x1 bounded error (0.2%*0.03863890967855284),x2 with error (0.2%*0.9613610903214472) : 0.9751368950717774
BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9895428015564203


In [39]:
#using knn with brute force from 15-360
for i in range(25):
    df_2feature_be = df_2feature_be_generates(BE,i,0)
    x_be = df_2feature_be.iloc[:, :-1].values
    y_be = df_2feature_be.iloc[:,-1].values
    X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
    KNN_BE = KNeighborsClassifier()
    KNN_BE.fit(X_train,y_train)
    y_BE_predict = KNN_BE.predict(X_test)
    result_BE_train = KNN_BE.score(X_train, y_train)
    result_BE_test = KNN_BE.score(X_test, y_test)
    subset_raw = KNN_BE.score(subsample_x, subsample_y)
    
    print('BEKNN Accuracy (on training) = with x1 bounded error ('+str(BE)+'%*'+str((error_a[i]/(error_a[i]+error_b[i])))+'),x2 with error ('+str(BE)+'%*'+str((error_b[i]/(error_a[i]+error_b[i])))+') : '+str(result_BE_train))
    print('BEKNN Accuracy (on testing) = with x1 bounded error ('+str(BE)+'%*'+str((error_a[i]/(error_a[i]+error_b[i])))+'),x2 with error ('+str(BE)+'%*'+str((error_b[i]/(error_a[i]+error_b[i])))+') : '+str(result_BE_test))
    print('BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error '+str(subset_raw))
    

d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


BEKNN Accuracy (on training) = with x1 bounded error (0.2%*0.0),x2 with error (0.2%*1.0) : 0.9759496793290577
BEKNN Accuracy (on testing) = with x1 bounded error (0.2%*0.0),x2 with error (0.2%*1.0) : 0.9721331689272503
BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9892996108949417


d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


BEKNN Accuracy (on training) = with x1 bounded error (0.2%*0.041666666666666664),x2 with error (0.2%*0.9583333333333334) : 0.9760113468179575
BEKNN Accuracy (on testing) = with x1 bounded error (0.2%*0.041666666666666664),x2 with error (0.2%*0.9583333333333334) : 0.9711467324290999
BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9883268482490273


d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


...

BEKNN Accuracy (on training) = with x1 bounded error (0.2%*0.9166666666666666),x2 with error (0.2%*0.08333333333333333) : 0.9789713862851505
BEKNN Accuracy (on testing) = with x1 bounded error (0.2%*0.9166666666666666),x2 with error (0.2%*0.08333333333333333) : 0.9721331689272503
BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9897859922178989


d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


BEKNN Accuracy (on training) = with x1 bounded error (0.2%*0.9583333333333334),x2 with error (0.2%*0.041666666666666664) : 0.9795263936852492
BEKNN Accuracy (on testing) = with x1 bounded error (0.2%*0.9583333333333334),x2 with error (0.2%*0.041666666666666664) : 0.9723797780517879
BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9890564202334631


d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


BEKNN Accuracy (on training) = with x1 bounded error (0.2%*1.0),x2 with error (0.2%*0.0) : 0.9808830784410458
BEKNN Accuracy (on testing) = with x1 bounded error (0.2%*1.0),x2 with error (0.2%*0.0) : 0.974845869297164
BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9900291828793775


In [38]:
light_CScore = 1.0117280381068128
CO2_CSore = 0.9885700987654558
ConfidenceSore_lightCO2 = [light_CScore/(light_CScore+CO2_CSore),CO2_CSore/(light_CScore+CO2_CSore)]

#using confidence as error allocation rate to predict by svm
df_2feature_be = df_2feature_be_generates(BE,-1,0,ConfidenceSore_lightCO2)
x_be = df_2feature_be.iloc[:, :-1].values
y_be = df_2feature_be.iloc[:,-1].values
X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
clf_BE = SVC(kernel='linear',C=1,gamma='auto')
clf_BE.fit(X_train, y_train)
y_BE_predict = clf_BE.predict(X_test)
result_BE_train = clf_BE.score(X_train, y_train)
result_BE_test = clf_BE.score(X_test, y_test)
subset_raw = clf_BE.score(subsample_x, subsample_y)
print('BESVM Accuracy (on training) using confidence score on x1 bounded error ('+str(BE)+'%*'+str(ConfidenceSore_lightCO2[1])+'),x2 with error ('+str(BE)+'%*'+str(ConfidenceSore_lightCO2[0])+') : '+str(result_BE_train))
print('BESVM Accuracy (on testing) = using confidence score on  x1 bounded error ('+str(BE)+'%*'+str(ConfidenceSore_lightCO2[1])+'),x2 with error ('+str(BE)+'%*'+str(ConfidenceSore_lightCO2[0])+') : '+str(result_BE_test))
print('BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error '+str(subset_raw))




d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to b

BESVM Accuracy (on training) using confidence score on x1 bounded error (0.2%*0.4942113780654799),x2 with error (0.2%*0.5057886219345201) : 0.9696610922006808
BESVM Accuracy (on testing) = using confidence score on  x1 bounded error (0.2%*0.4942113780654799),x2 with error (0.2%*0.5057886219345201) : 0.9704010655616397
BESVM Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9873540856031129


In [37]:
#using confidence as error allocation rate to predict by knn
df_2feature_be = df_2feature_be_generates(BE,-1,0,ConfidenceSore_lightCO2)
x_be = df_2feature_be.iloc[:, :-1].values
y_be = df_2feature_be.iloc[:,-1].values
X_train,X_test,y_train, y_test = train_test_split(x_be, y_be, test_size=0.2,random_state=1)
KNN_BE = KNeighborsClassifier()
KNN_BE.fit(X_train, y_train)
y_BE_predict = KNN_BE.predict(X_test)
result_BE_train = KNN_BE.score(X_train, y_train)
result_BE_test = KNN_BE.score(X_test, y_test)
subset_raw = KNN_BE.score(subsample_x, subsample_y)
print('BEKN Accuracy (on training) using confidence score on x1 bounded error ('+str(BE)+'%*'+str(ConfidenceSore_lightCO2[1])+'),x2 with error ('+str(BE)+'%*'+str(ConfidenceSore_lightCO2[0])+') : '+str(result_BE_train))
print('BEKNN Accuracy (on testing) = using confidence score on  x1 bounded error ('+str(BE)+'%*'+str(ConfidenceSore_lightCO2[1])+'),x2 with error ('+str(BE)+'%*'+str(ConfidenceSore_lightCO2[0])+') : '+str(result_BE_test))
print('BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error '+str(subset_raw))


d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\BESVM\BESVM\.venv\lib\site-packages\ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to b

BEKNN Accuracy (on training) = with x1 bounded error (0.2%*0.03863890967855284),x2 with error (0.2%*0.9613610903214472) : 0.9764318484534557
BEKNN Accuracy (on testing) = with x1 bounded error (0.2%*0.03863890967855284),x2 with error (0.2%*0.9613610903214472) : 0.9718810122835578
BEKNN Accuracy (on subset_raw) = with no bounded error,x2 with no error 0.9880836575875487
